In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [2]:
#данные по уникальным точкам
uniq_point = pd.read_csv('data/uniq_point.csv', sep = ',')
uniq_point.head(2)

,ap_mac,lat,lon,Субъект
0,3420E31780D0,55.721146,37.606163,Якиманка
1,8CFE74212CD0,55.737167,37.623783,Якиманка


In [3]:
#наши точки
X_zone = uniq_point[['lat','lon']].values

In [4]:
#функция для нахождения блюжайших точек в радиусе
def r_neighbors(X_train, r=0.01):
    #0.01 - 1 км
    neigh = NearestNeighbors(n_neighbors=10, radius=r)
    neigh.fit(X_train)
    distances, indexes = neigh.radius_neighbors(X_zone, r)
    return np.array([len(x) for x in indexes])

In [5]:
#туалеты
toilet = pd.read_csv('геоданные/842_norm/842.csv', sep = ';', index_col=0)
toilet.columns = ['ADDRESS', 'REGIME', 'AVAILABIL', 'LON', 'LAT', 'GLOBALID']
toilet['LON'] = toilet['LON'].str.replace("'", "").str.replace(",", ".").astype(float)
toilet['LAT'] = toilet['LAT'].str.replace("'", "").str.replace(",", ".").astype(float)
#количество туалетов в километровом радуиусе
X_train = toilet[['LAT', 'LON']].values
uniq_point['toilet'] = r_neighbors(X_train, r=0.01)

In [6]:
#торговля и общепит стационарные
riteil = pd.read_csv('геоданные/586_norm/586.csv', sep = ';', index_col=0)
riteil.columns = ['NAME', 'ADDRESS', 'lat', 'lon', 'NAME_OPF', 'LIC_SERNUM', 'LIC_START', 'LIC_STOP','LIC_STATUS']
#количество торговых точек и общепита в радиусе 1 км
X_train = riteil[['lat', 'lon']].values
uniq_point['riteil'] = r_neighbors(X_train, r=0.01)

In [7]:
#бытовые услуги
service = pd.read_excel('геоданные/быт_усл-4272-2021-06-09.xlsx')
#количество точек бытовых в радиусе 1 км
X_train = service[['Latitude_WGS84', 'Longitude_WGS84']].values
uniq_point['service'] = r_neighbors(X_train, r=0.01)

In [8]:
#станции метро
metro = pd.read_csv('геоданные/624_norm/624.csv', sep = ';', index_col=0)
#количество станций метро в радиусе 1 км
X_train = metro[['LAT', 'LON']].values
uniq_point['metro'] = r_neighbors(X_train, r=0.01)

In [9]:
#объекты для аренды
arenda = pd.read_csv('геоданные/1113_norm/1113.csv', sep = ';', index_col=0)
#количество свободных помещений в радиусе 5 км
X_train = arenda[['LAT', 'LON']].values
uniq_point['arenda'] = r_neighbors(X_train, r=0.05)

In [10]:
#клубы
club = pd.read_csv('геоданные/493_norm/493.csv', sep = ';', index_col=0)
#количество клубов в радиусе 2 км
X_train = club[['LAT', 'LON']].values
uniq_point['club'] = r_neighbors(X_train, r=0.02)

In [11]:
#больницы
med = pd.read_csv('геоданные/502_norm/502.csv', sep = ';', index_col=0)
#количество больниц в радиусе 5 км
X_train = med[['LAT', 'LON']].values
uniq_point['med'] = r_neighbors(X_train, r=0.05)

In [12]:
#ветбольницы
vet = pd.read_csv('геоданные/1193_norm/1193.csv', sep = ';', index_col=0)
vet['LON'] = vet['LON'].str.replace("'", "").str.replace(",", ".").astype(float)
vet['LAT'] = vet['LAT'].str.replace("'", "").str.replace(",", ".").astype(float)
#количество ветлечебниц в радиусе 5 км
X_train = vet[['LAT', 'LON']].values
uniq_point['vet'] = r_neighbors(X_train, r=0.05)

In [13]:
#стоматологи
stomat = pd.read_csv('геоданные/518_norm/518.csv', sep = ';', index_col=0)
#количество стоматологий в радиусе 5 км
X_train = stomat[['LAT', 'LON']].values
uniq_point['stomat'] = r_neighbors(X_train, r=0.05)

In [14]:
uniq_point.head(2)

,ap_mac,lat,lon,Субъект,toilet,riteil,service,metro,arenda,club,med,vet,stomat
0,3420E31780D0,55.721146,37.606163,Якиманка,2,51,80,2,3,1,2,4,8
1,8CFE74212CD0,55.737167,37.623783,Якиманка,2,147,132,4,2,3,2,3,8


In [15]:
uniq_point[['ap_mac', 'toilet', 'metro', 'arenda', 'riteil', 'service', 'club', 'med', 'vet', 'stomat']].to_csv('data/df_geo.csv', index=False)